In [1]:
import pandas as pd
import numpy as np
import pyrankvote
from pyrankvote import Candidate, Ballot
from IPython.display import Image

df = pd.read_csv("/Users/anxhela/Downloads/election_04-27-2020_13-25-23_[1]cands_[2]noise.csv")

In [2]:
df.head()

,Unnamed: 0,candidate_1,candidate_2,candidate_3
0,0,candidate_1,candidate_2,candidate_3
1,1,candidate_1,candidate_2,candidate_3
2,2,candidate_1,candidate_2,candidate_3
3,3,candidate_1,candidate_2,candidate_3
4,4,candidate_1,candidate_2,candidate_3


In [8]:
# Candidates:

can_one = Candidate('candidate_1')
can_two = Candidate('candidate_2')
can_three = Candidate('candidate_3')

candidates = [can_one, can_two, can_three]

In [11]:
list_of_ballot_choices = df[["candidate_1","candidate_2","candidate_3"]].values.tolist()

In [12]:
for x in list_of_ballot_choices:
    while '0' in x:
        x.remove('0')

In [13]:
df['ballot_choices_list'] = list_of_ballot_choices
df.head()

,Unnamed: 0,candidate_1,candidate_2,candidate_3,ballot_choices_list
0,0,candidate_1,candidate_2,candidate_3,"[candidate_1, candidate_2, candidate_3]"
1,1,candidate_1,candidate_2,candidate_3,"[candidate_1, candidate_2, candidate_3]"
2,2,candidate_1,candidate_2,candidate_3,"[candidate_1, candidate_2, candidate_3]"
3,3,candidate_1,candidate_2,candidate_3,"[candidate_1, candidate_2, candidate_3]"
4,4,candidate_1,candidate_2,candidate_3,"[candidate_1, candidate_2, candidate_3]"


In [14]:
def remove_anomalies_from_ballots(ballots, value_to_remove):
    """
    ballots: series from dataframe. Each row contains a list of ranked candidates representing 1 ballot.
    value_to_remove: string that a ballot contains as a condition for its deletion
    """
    indices_to_drop = []
    
    for index, value in enumerate(ballots):
        if value_to_remove in value:
            indices_to_drop.append(index)
                   
    return indices_to_drop

In [16]:
remove_anomalies_from_ballots(df, "0")

[0]

In [18]:
df.shape

(37516, 5)

In [21]:

# creating a ballot from ballot_choice_list

candidates_array = []

for candidate_list in df['ballot_choices_list']:
    ballot = []
    for candidate in candidate_list:
        candidate = candidate.strip(' ')
        ballot.append(Candidate(candidate))
    candidates_array.append(ballot)
candidates_array

[[<Candidate('candidate_1')>,
  <Candidate('candidate_2')>,
  <Candidate('candidate_3')>],
 [<Candidate('candidate_1')>,
  <Candidate('candidate_2')>,
  <Candidate('candidate_3')>],
 [<Candidate('candidate_1')>,
  <Candidate('candidate_2')>,
  <Candidate('candidate_3')>],
 [<Candidate('candidate_1')>,
  <Candidate('candidate_2')>,
  <Candidate('candidate_3')>],
 [<Candidate('candidate_1')>,
  <Candidate('candidate_2')>,
  <Candidate('candidate_3')>],
 [<Candidate('candidate_1')>,
  <Candidate('candidate_2')>,
  <Candidate('candidate_3')>],
 [<Candidate('candidate_1')>,
  <Candidate('candidate_2')>,
  <Candidate('candidate_3')>],
 [<Candidate('candidate_1')>,
  <Candidate('candidate_2')>,
  <Candidate('candidate_3')>],
 [<Candidate('candidate_1')>,
  <Candidate('candidate_2')>,
  <Candidate('candidate_3')>],
 [<Candidate('candidate_1')>,
  <Candidate('candidate_2')>,
  <Candidate('candidate_3')>],
 [<Candidate('candidate_1')>,
  <Candidate('candidate_2')>,
  <Candidate('candidate_3')>],

In [23]:
df['ballot_choices_list_with_candidate_objects'] = candidates_array

In [24]:
#using pyrank's Ballot function to create ballots from the ballot choice object column

ballot_objects = []

for index,value in enumerate(df['ballot_choices_list_with_candidate_objects']):
    ballot = Ballot(ranked_candidates=value)
    ballot_objects.append(ballot)
ballot_objects

[<Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(candidate_2, candidate_3)>,
 <Ballot(candidate_1, candidate_2, candidate_3)>,
 <Ballot(cand

In [26]:
df['ballot_objects'] = ballot_objects


In [29]:
election_result = pyrankvote.instant_runoff_voting(candidates, df['ballot_objects'])
election_result

ElectionResults(3 rounds)

In [30]:
print(election_result)

ROUND 1
Candidate      Votes  Status
-----------  -------  --------
candidate_1    15145  Hopeful
candidate_3    11332  Hopeful
candidate_2    11039  Hopeful

ROUND 2
Candidate      Votes  Status
-----------  -------  --------
candidate_1    18928  Hopeful
candidate_3    18588  Hopeful
candidate_2        0  Rejected

FINAL RESULT
Candidate      Votes  Status
-----------  -------  --------
candidate_1    37516  Elected
candidate_3        0  Rejected
candidate_2        0  Rejected



In [31]:
#IRV Winner:
winners = election_result.get_winners()

In [33]:
#Condorcet winner:

def create_ballot_dict(df):
    all_ballots = list(df['ballot_objects'].values)
    ballot_dict={}
    for i in range(len(all_ballots)):
        ballot_str=str(all_ballots[i])
        curr_ballot = all_ballots[i]
        if ballot_str in ballot_dict:
            ballot_dict[ballot_str][0]=ballot_dict[ballot_str][0]+1
        else:
            ballot_dict[ballot_str]=[1,all_ballots[i]]
    return ballot_dict

def create_candidate_matrix(candidates):
    candidate_names=[]
    for cand in candidates:
        candidate_names.append(cand.name)
        
    print("candidate_names: ",candidate_names)
    cand_matrix= pd.DataFrame(0, columns=candidate_names, index=candidate_names)
    print(cand_matrix)
    return cand_matrix

def symbolize_cand_matrix(cand_matrix):
    for i in cand_matrix.index:
        for j in cand_matrix.columns:
            if i ==j:
                cand_matrix.loc[i,j] ='`'
            else:
                if cand_matrix.loc[i,j]!= '++' and cand_matrix.loc[i,j]!= '--':
                    if cand_matrix.loc[i,j]>cand_matrix.loc[j,i]:
                        cand_matrix.loc[i,j] = '++'
                        cand_matrix.loc[j,i] = '--'
                    else:
                        cand_matrix.loc[i,j] = '--'
                        cand_matrix.loc[j,i] = '++'
    print(cand_matrix)

def countX(lst, x): 
    return lst.count(x) 

def return_winners(cand_matrix):

    result_dict={}
    for i in cand_matrix.index:
        key = ''.join([str(countX(list(cand_matrix.loc[i]),'--')+1), ". ",i])
        result_dict[key]= (countX(list(cand_matrix.loc[i]),'++'), countX(list(cand_matrix.loc[i]),'--'))
    for key in sorted(result_dict.keys()): 
        print("Rank %s: (Wins, losses) %s" % (key, result_dict[key]))

In [34]:
##Condorcet_winner returns the Condorcet results
##Input params: Dataframe with ballot_objects column and the list of candidates

def Condorcet_winner(df, candidates):
    all_ballots = list(df['ballot_objects'].values)
    print("Creating the ballot dictionary...")
    ballot_dict=create_ballot_dict(df)
    print("Creating the candidate matrix to store pairwise results...")
    candidate_names=[]
    for cand in candidates:
        candidate_names.append(cand.name)
        
    print("\ncandidate_names: ",candidate_names)
    cand_matrix= pd.DataFrame(0, columns=candidate_names, index=candidate_names)
    print("\nUpdating the candidate matrix with pairwise results...")

    for ballot in ballot_dict:
        ranked_candidates = list(ballot_dict[ballot][1].ranked_candidates)
        for i in range(len(ranked_candidates)):
            for j in range(len(ranked_candidates)):
                if (ranked_candidates[i].name!=ranked_candidates[j].name) & (i<j):
                    cand_matrix[ranked_candidates[j].name][ranked_candidates[i].name] = cand_matrix[ranked_candidates[j].name][ranked_candidates[i].name]+ballot_dict[ballot][0]
    
    print("\n",cand_matrix)
    print("\nSymbolizing the candidate matrix with pairwise results...\n")

    symbolize_cand_matrix(cand_matrix)
    print("\n\nCondorcet results: ")
    return_winners(cand_matrix)

In [35]:
Condorcet_winner(df, candidates)

Creating the ballot dictionary...
Creating the candidate matrix to store pairwise results...

candidate_names:  ['candidate_1', 'candidate_2', 'candidate_3']

Updating the candidate matrix with pairwise results...

              candidate_1  candidate_2  candidate_3
candidate_1            0        25112        18649
candidate_2        11902            0        18643
candidate_3        18348        18333            0

Symbolizing the candidate matrix with pairwise results...

            candidate_1 candidate_2 candidate_3
candidate_1           `          ++          ++
candidate_2          --           `          ++
candidate_3          --          --           `


Condorcet results: 
Rank 1. candidate_1: (Wins, losses) (2, 0)
Rank 2. candidate_2: (Wins, losses) (1, 1)
Rank 3. candidate_3: (Wins, losses) (0, 2)
